## News :D



In [2]:
%load_ext autoreload
%autoreload 2

import requests
import re
import itertools
from bs4 import BeautifulSoup

Installs:
lxmn
bs4

In [11]:
months = ['January', 'February','March','April','May','June','July','August','September','October','November','December']


# CNN                                  has all articles
cnn_dict = dict([(i, [f'https://www.cnn.com/article/sitemap-{i}-{j}.html' for j in range(1,13)]) for i in range(2014,2025)])


# New York Times                       has days
nyt_dict = dict([(i, [f'https://www.nytimes.com/sitemap/{i}/{str(j).zfill(2)}/' for j in range(1,13)]) for i in range(2014,2025)])


# Business Insider                     has all articles
bi_dict = dict([(i, [f'https://www.businessinsider.com/sitemap/html/{i}-{str(j).zfill(2)}.html' for j in range(1,13)]) for i in range(2014,2025)])


# The Hill                             has days
th_dict = dict([(i, [f'https://thehill.com/sitemap/{i}/{j}/' for j in months]) for i in range(2014,2025)])


# New York Post                        has days
nyp_dict = dict([(i, [f'https://nypost.com/{i}/{str(j).zfill(2)}/' for j in range(1,13)]) for i in range(2014,2025)])


# Daily Mail                           has days
dm_dict = dict([(i, [f'https://www.dailymail.co.uk/home/sitemaparchive/month_{i}{str(j).zfill(2)}.html' for j in range(1,13)]) for i in range(2014,2025)])


# Fox
fox_links = ['https://www.foxnews.com/sitemap.xml?type=articles'] + [f'https://www.foxnews.com/sitemap.xml?type=articles&page={i}' for i in range(2, 161)]

In [12]:
print(cnn_dict[2024])
print(nyt_dict[2024])
print(bi_dict[2024])
print(th_dict[2024])
print(nyp_dict[2024])
print(dm_dict[2024])

['https://www.cnn.com/article/sitemap-2024-1.html', 'https://www.cnn.com/article/sitemap-2024-2.html', 'https://www.cnn.com/article/sitemap-2024-3.html', 'https://www.cnn.com/article/sitemap-2024-4.html', 'https://www.cnn.com/article/sitemap-2024-5.html', 'https://www.cnn.com/article/sitemap-2024-6.html', 'https://www.cnn.com/article/sitemap-2024-7.html', 'https://www.cnn.com/article/sitemap-2024-8.html', 'https://www.cnn.com/article/sitemap-2024-9.html', 'https://www.cnn.com/article/sitemap-2024-10.html', 'https://www.cnn.com/article/sitemap-2024-11.html', 'https://www.cnn.com/article/sitemap-2024-12.html']
['https://www.nytimes.com/sitemap/2024/01/', 'https://www.nytimes.com/sitemap/2024/02/', 'https://www.nytimes.com/sitemap/2024/03/', 'https://www.nytimes.com/sitemap/2024/04/', 'https://www.nytimes.com/sitemap/2024/05/', 'https://www.nytimes.com/sitemap/2024/06/', 'https://www.nytimes.com/sitemap/2024/07/', 'https://www.nytimes.com/sitemap/2024/08/', 'https://www.nytimes.com/sitema